<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Modeling" data-toc-modified-id="Modeling-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modeling</a></span></li><li><span><a href="#Negative-Sampling" data-toc-modified-id="Negative-Sampling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Negative Sampling</a></span></li></ul></div>

In [ ]:
# for col, z in inputs.items():
#     print(col, z)
#     embedding_mlr_list[col] = Embedding(input_dim=data[col].nunique(), output_dim=1, name=f"Embedding_{col}")(z)

# embedding_mlr_list = []
# for col in data.columns:
#     if col != "cnt":
#         embedding_mlr_list.append(Embedding(input_dim= data[col].nunique(), output_dim=1, name=f"{col}_lr_emb")(inputs[col]))

# class LR(Layer):
#     def build(self, input_shape):
#         # 레이어 연결시 0으로 초기화된 학습 가중치 생성
#         self.b = self.add_weight(shape=(1, ), initializer="zeros", trainable=True)
#         super().build(input_shape)
    
#     def call(self, inputs, **kwargs):
#         logits = tf.add_n(inputs) + self.b
#         return logits

# # Order-2 Part
# dim = 8
# embedding_fm_list = []
# for col in data.columns:
#     if col != "cnt":
# #         fm_embedding_mlr_list.append(Embedding(input_dim= data[col].nunique(), output_dim=dim, name=f"{col}_fm_emb")(inputs[col]))
#         print(Embedding(input_dim= data[col].nunique(), output_dim=dim, name=f"{col}_fm_emb").get_weights)

In [69]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Layer, Dense, Flatten, Dropout, Concatenate, Add, Dot, Multiply, Reshape, Activation, BatchNormalization, SimpleRNNCell, RNN, SimpleRNN, LSTM, Embedding, Bidirectional, TimeDistributed, Conv1D, Conv2D, MaxPool1D, MaxPool2D, GlobalMaxPool1D, GlobalMaxPool2D, AveragePooling1D, AveragePooling2D, GlobalAveragePooling1D, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import SGD, Adam, Adagrad
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, CosineSimilarity
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.activations import linear, sigmoid, relu
from tensorflow.keras.initializers import RandomNormal, glorot_uniform, he_uniform, Constant
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import seaborn as sb
import sklearn
from sklearn.model_selection import train_test_split
import random
from tqdm.notebook import tqdm

plt.style.use("dark_background")

In [46]:
data = pd.read_csv("./Datasets/Frappe/frappe.csv")
metadata = pd.read_csv("./Datasets/Frappe/frappe_meta.csv", sep="\t")

# "특정 상황에서 고객이 한번 쯤 사용해볼 것 같은 어플을 찾는 추천시스템"으로 목표를 설정해 보겠습니다. 즉, 특정 상황에서 유저가 한번도 사용하지 않았던 어플 중 가장 관심있어 할만한 어플을 찾아주는 문제로 정의할 수 있습니다.
# 1번 이상 로그가 있는 Context에 대해서는 모두 "클릭을 했다" 라는 의미로 `True`로 변경해줍니다.
# data["cnt"] = True

# Modeling

In [35]:
output_dim = 8
inputs = dict()
embedding_mlr_list = list()
embedding_fm_list = list()
for col in data.columns:
    if col != "cnt":
        input_dim = data[col].nunique()
        z = Input(shape=(), name=f"Input_{col}")
        inputs[col] = z

        # Order-1 Part
        z1 = Embedding(input_dim=input_dim, output_dim=1, name=f"Embedding_mlr_{col}")(z)
        embedding_mlr_list.append(z1)
        
        # Order-2 Part
        z2 = Embedding(input_dim=input_dim, output_dim=output_dim, name=f"Embedding_fm_{col}")(z)
        embedding_fm_list.append(z2)
logits_mlr = tf.Variable((0.,)) + tf.math.add_n(embedding_mlr_list)

# Order-2 Part
z = tf.stack(embedding_fm_list, axis=1)
logits_fm = 1/2*tf.math.reduce_sum(tf.math.square(tf.math.reduce_sum(z, axis=1)) - tf.math.reduce_sum(tf.math.square(z), axis=1), axis=1, keepdims=True)

# Order-3 Part
z = Concatenate(axis=1)(embedding_fm_list)
z = Dense(units=100, activation="relu")(z)
z = Dropout(rate=0.5)(z)
z = Dense(units=100, activation="relu")(z)
z = Dropout(rate=0.5)(z)
z = Dense(units=100, activation="relu")(z)
z = Dropout(rate=0.5)(z)
logits_mlp = Dense(units=1)(z)

z = Add()([logits_mlr, logits_fm, logits_mlp])
logits = sigmoid(z)
model = Model(inputs=inputs, outputs=logits, name="deepfm")

model.compile(optimizer=Adagrad(), loss="binary_crossentropy", metrics=["binary_accuracy"])
model.summary()

Model: "deepfm"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input_user (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 Input_item (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 Input_daytime (InputLayer)     [(None,)]            0           []                               
                                                                                                  
 Input_weekday (InputLayer)     [(None,)]            0           []                               
                                                                                             

In [43]:
for col in data.columns:
    data[col] = pd.Categorical(data[col])
    data[col] = data[col].cat.codes

# cat2idx = {}
# cat2size = {}
# for col in data.columns:
#     if col != "cnt":
#         data[col] = data[col].astype("category")
        
#         cat2idx[col] = {cat:idx for idx, cat in enumerate(data[col].cat.categories)}
#         cat2size[col] = len(data[col].cat.categories)

# for col, dic in cat2idx.items():
#     data[col] = data[col].map(dic)

tr, te = train_test_split(data, test_size=0.1, shuffle=True)

# Negative Sampling
- 현재 해당 데이터는 특정 컨택스트에서 고객의 앱에 대한 사용 여부로서 "Positive" 데이터만 존재하고 있습니다. 즉, 고객 행동의 긍정적인 행동만 기록되어있다는 것입니다. 우리는 고객이 클릭하지 않은 앱에 대한 사용 여부, 즉 Positive 데이터 한개당 Negative 데이터 3개씩을 생성하되 item 이외의 다른 feature는 고정하고 item을 무작위로 결정하는 식으로 진행해보도록 하겠습니다.

In [47]:
data.head()

,user,item,daytime,weekday,isweekend,homework,cost,weather,country,city,cnt
0,0,0,morning,sunday,weekend,unknown,free,sunny,United States,0,1
1,0,9,afternoon,friday,workday,unknown,free,cloudy,United States,1023,2
2,0,9,evening,friday,weekend,unknown,free,cloudy,United States,1023,9
3,0,14,afternoon,friday,workday,unknown,free,cloudy,United States,1023,2
4,0,16,evening,friday,weekend,unknown,free,cloudy,United States,1023,1


In [ ]:
# item_cnt = data.groupby(["item"])["cnt"].sum()
# random.choices(item_cnt.index, k=len(data)*3, weights=item_cnt.values)

In [74]:
item_cnt = data.groupby(["item"])["cnt"].sum()
rows = list()
for name, row in tqdm(data.iloc[:100].iterrows()):
    cnt = 0
    while cnt != 3:
        neg_item = random.choices(item_cnt.index, k=1, weights=item_cnt.values)[0]
        if row["item"] != neg_item:
            row["item"] = neg_item
            row["cnt"] = False
            rows.append(row)
            cnt += 1
#         else:
#             continue

In [99]:
temp = pd.concat([data]*3).iloc[:100]
item_cnt = data.groupby(["item"])["cnt"].sum()
for name, row in temp.iterrows():
    neg_item = random.choices(item_cnt.index, k=1, weights=item_cnt.values)[0]
    if row["item"] != neg_item:
        row["item"] = neg_item
        row["cnt"] = False

In [100]:
temp

,user,item,daytime,weekday,isweekend,homework,cost,weather,country,city,cnt
0,0,0,morning,sunday,weekend,unknown,free,sunny,United States,0,1
1,0,9,afternoon,friday,workday,unknown,free,cloudy,United States,1023,2
2,0,9,evening,friday,weekend,unknown,free,cloudy,United States,1023,9
3,0,14,afternoon,friday,workday,unknown,free,cloudy,United States,1023,2
4,0,16,evening,friday,weekend,unknown,free,cloudy,United States,1023,1
...,...,...,...,...,...,...,...,...,...,...,...
95,1,5,afternoon,tuesday,workday,unknown,free,sunny,Spain,0,1
96,1,5,afternoon,tuesday,workday,unknown,free,sunny,Spain,369,114
97,1,5,afternoon,wednesday,workday,unknown,free,cloudy,Spain,369,11
98,1,5,afternoon,wednesday,workday,unknown,free,sunny,Spain,369,126


In [184]:
def negative_sampling(df, ratio=3):
    neg_sampling = pd.concat([df]*ratio)
    neg_sampling["item"] = df["item"].sample(n=len(neg_sampling), replace=True).values
    # "네거티브 데이터"는 카운트가 존재하지 않음
    neg_sampling["cnt"] = 0

    sampling = pd.concat([df, neg_sampling])
    sampling = sampling.drop_duplicates(sampling.columns.drop("cnt"))
    return sampling

In [185]:
train_sampling = negative_sampling(train)
test_sampling = negative_sampling(test)

- (3) Input 과 Target 분리
- 데이터 중에서 종속 변수가 되는 요소들( X )과 독립 변수가 되는 클릭 유무을 나누도록 하겠습니다.

In [187]:
train_x = train.drop(["cnt"], axis=1)
train_y = train[["cnt"]]

test_x = test.drop(["cnt"], axis=1)
test_y = test[["cnt"]]

- (4) 텐서로 바꾸어 주기

In [207]:
batch_size = 512

train_set = tf.data.Dataset.from_tensor_slices(({k:v.values.astype("int16") for k, v in train_x.iteritems()}, train_y.values)).shuffle(len(train_x)).batch(batch_size, drop_remainder=True)
test_set = tf.data.Dataset.from_tensor_slices(({k:v.values.astype("int16") for k, v in test_x.iteritems()}, test_y.values)).shuffle(len(test_x)).batch(batch_size, drop_remainder=True)

In [208]:
train_set

<BatchDataset shapes: ({user: (512,), item: (512,), daytime: (512,), weekday: (512,), isweekend: (512,), homework: (512,), cost: (512,), weather: (512,), country: (512,), city: (512,)}, (512, 1)), types: ({user: tf.int16, item: tf.int16, daytime: tf.int16, weekday: tf.int16, isweekend: tf.int16, homework: tf.int16, cost: tf.int16, weather: tf.int16, country: tf.int16, city: tf.int16}, tf.int64)>

In [211]:
train_x

,user,item,daytime,weekday,isweekend,homework,cost,weather,country,city
38883,119,56,3,3,0,1,0,8,75,0
35512,104,19,1,4,1,1,0,0,75,181
80882,355,506,0,3,0,1,0,7,55,148
20930,64,190,1,0,0,0,0,0,17,0
93222,770,283,2,5,1,1,0,7,75,0
...,...,...,...,...,...,...,...,...,...,...
31665,90,23,3,2,0,1,0,3,12,0
89659,501,462,0,2,0,1,0,0,75,173
15745,48,36,2,1,1,1,1,7,67,0
43259,133,100,4,1,1,2,0,0,75,0
